**Función:** crea ndarray con versiones 224x224 de las fotos del dsub dataset 10k.

**Requiere:**
- ../datasets/df_clean_run1.json
- ../datasets/photos (con todas las fotos del dataset)

**Crea:**
(demo, no genera output)

In [10]:
import pandas as pd
import numpy as np
import shutil
import math
import torchvision.transforms as T
from tqdm.notebook import trange, tqdm
from keras.utils import np_utils
from PIL import Image

In [ ]:
ejecutadoEnGoogleDrive = False # True si se ejecuta en Google Drive

In [ ]:
if ejecutadoEnGoogleDrive:
    from google.colab import drive
    drive.mount('/content/drive')
    PATH = '/content/drive/clasificacion_imagenes'
else:
    PATH = '../'

# Create Train Test DFs. 

In [11]:
df = pd.read_json(PATH + "datasets/df_clean.json")

In [12]:
df.columns=['photo_id', 'label']

In [13]:
df.head()

,photo_id,label
0,fB-J1F60DgMRoc8eI0z0Ew,food
1,n2hdfzYgxL0twgfSDbro1w,food
2,Z77aLYI_SvAl7PaJ62umJQ,food
3,tBdw-FR8aNZcJLa5bX-SdA,food
4,z-ftfswcyAAERjoMzKFJ5A,food


In [14]:
df.label.value_counts()

food       4786
inside     2150
drink       677
outside     611
menu         88
Name: label, dtype: int64

In [15]:
round(df.label.value_counts() * 0.8, 0)

food       3829.0
inside     1720.0
drink       542.0
outside     489.0
menu         70.0
Name: label, dtype: float64

In [11]:
def create_df(num_samples, dataframe):
    labels = list(df.label.value_counts().reset_index()['index'])
    proportion = list(df.label.value_counts() / df.shape[0])
    
    dict_samples = {}
    
    for (i, j) in zip(labels, proportion):
        dict_samples[i] = int(round(j * num_samples, 0))

    df_short = pd.DataFrame()

    for i in dict_samples.keys():
        df_temp = dataframe[dataframe['label'] == i].sample(dict_samples[i])
        df_short = pd.concat([df_short, df_temp], axis=0)

    return df_short


In [12]:
train_num = round(df.shape[0] * 0.8, 0)
df_train = create_df(train_num, df)

In [13]:
df_train.reset_index(drop=True, inplace=True)
df_train.tail()

,photo_id,label
6559,v1brBS7juhaP1YkXLp7ZQA,menu
6560,ockFtmG4btt1bf9GgVRNSg,menu
6561,86f7GiUFQsOtZ6_KZaJLYw,menu
6562,rv8yDNDmK6YibyKQMtUp7g,menu
6563,P2H9Uyem-pc1b2WUn6F24Q,menu


In [14]:
train_ids = list(df_train.photo_id)
print(train_ids)

['tklQZg27obBW8ooyPg6oDA', '31ZoDyh_pUNJdYoiXou1NA', 'Sk3iE11HO5XHAv5LFHp3NA', 'mgOuyb26kSYGfqmOnSswqA', 'bYVCG_M51tuyBXVNqIqn8g', '3HIDw67-_BMp_U7fXbIfzw', '4SGHzIDFqneNz28xY2QDIg', '-bdOahUmzaI2v4gPc9lA0Q', '5NaJrw2cbzjF4LC1ssLKmQ', 'rRMjyEmsacPuhlDPpgr74Q', '4tH0fMB_hYWN058nPKd7mg', 'W1Hdm6N9KEkgVf1fzwj5Qw', '7ZR1zKDUBK2Y-vaTLFXHCw', 'R0pC1fXUkD1Xic7Faxl4wg', '0OXE76Udfeww_2IkeFR63A', 'c0-xcXg2ZRpc8Wl2lQEjqg', 'nlnOaZTKY0KM7i1tW5Hu_Q', '--Kofko5jy33_vPJOEt4Ow', 'wF4NXkmcT40uaoesT4xipg', '9y9yDGVs1Tw4XQNL1hEOEg', 'jLuZPx2mSu_uHnWFN_xJ4Q', 'D3BL3ywAwnC6sHYRnYmf_A', 'm4yKuw1JiMm3njiwNyjLqg', 's9RUuimAM9B2RGLScPkW0g', 'GCd0IN30JOu-_bkBaskleg', 'DoQN1PBsZfjauLSCIidnVQ', 'xxRSxLozJF4ROpXNuu4k_w', '3HQNPlakdPdOY5Kbo68_VQ', 'vTtKt0vjO1zJsd1pw30--g', 'RK2meU8ubOTYGL6zlUxRLw', 'MJc0lBh7jY62Zrrp10DUpw', 'aabVv0L2LEKrJTkPZ0wL7g', 'a3uuBc96zf0DMO1RG22_Pg', 'PBLOR1dEJtrU3sYX6PsuVg', '6CR5lUPYGtOdCmcBthjVgQ', 'UX9UpOzu-K92n5Cj2SGvjg', 'ZizwKWzawgqzuw02DmpqxA', 'WRLHHqHVHRCnAlQgaUDZcg', 'tEYD9BckqQ

In [15]:
df_test = df[df['photo_id'].isin(train_ids) == False]
df_test.reset_index(drop=True, inplace=True)
df_test.tail()


,photo_id,label
1636,jJ1vIFzo-soZsS9ej2D8_A,menu
1637,UN08a0LgT7eFCAKXDnCXeQ,menu
1638,bz8LOw9gzmCrfXdfj87aPw,menu
1639,sCcfOgqPgs3j7u_l3FfrzQ,menu
1640,uHmi8LQMOeU9hNuuc6-pqg,menu


In [16]:
print(f'El tamaño de Train es de {df_train.shape[0]} y el de Test es de {df_test.shape[0]}')

El tamaño de Train es de 6564 y el de Test es de 1641


# Reescalar

In [26]:
def resize_pictures(dataframe, img_path, size_rescaled):
    df_temp = dataframe.copy()
    df_temp['image'] = ''
    
    for im_pos in tqdm(range(len(df_temp)), desc='Photos cropped', miniters=len(df_temp)//100):
        
        with Image.open('{}/{}.jpg'.format(img_path, df_temp.iloc[im_pos].photo_id)) as image:

            width = image.size[0]
            height = image.size[1]

            # redimensionar
            if (height < width): # imagen estrecha
                height = int(size_rescaled)
                height = math.floor(size_rescaled * width/height)
            else: # imagen ancha
                width = math.floor(size_rescaled * height/width)
                height = int(size_rescaled)

            resized = T.Resize((height, width))(image)
            cropped = T.CenterCrop(size=int(size_rescaled))(resized)
            df_temp.iloc[im_pos, 2] = cropped

    return df_temp

In [27]:
img_path = PATH + 'datasets/photos'
img_size = 224

# Create new train DF
df2_train = resize_pictures(df_train,
                             img_path,
                             img_size)

# Create new Test DF
df2_test = resize_pictures(df_test,
                             img_path,
                             img_size)

Photos cropped:   0%|          | 0/6564 [00:00<?, ?it/s]

Photos cropped:   0%|          | 0/1641 [00:00<?, ?it/s]

In [22]:
np.array(df2_train.iloc[1, 2]).shape

(224, 224, 3)

In [64]:
display(df2_train.head())
display(df2_test.head())

,photo_id,label,image
0,qMlh6WnM5I1acYuTeW9feQ,food,<PIL.Image.Image image mode=RGB size=224x224 a...
1,5s-aovv3OFbTwV04hbqAMQ,food,<PIL.Image.Image image mode=RGB size=224x224 a...
2,Dpslj0SNkln7Jgl4iNQoHQ,food,<PIL.Image.Image image mode=RGB size=224x224 a...
3,K_vRmofzBl-HaxihFQpWcA,food,<PIL.Image.Image image mode=RGB size=224x224 a...
4,-POC3Jknsac0ummGQ6Z6gw,food,<PIL.Image.Image image mode=RGB size=224x224 a...


,photo_id,label,image
0,fB-J1F60DgMRoc8eI0z0Ew,food,<PIL.Image.Image image mode=RGB size=224x224 a...
1,z-ftfswcyAAERjoMzKFJ5A,food,<PIL.Image.Image image mode=RGB size=224x224 a...
2,Pb1bW49aZ8xp85TmXP-jqQ,food,<PIL.Image.Image image mode=RGB size=224x224 a...
3,PelSOijELLLHADVzOBH-ag,food,<PIL.Image.Image image mode=RGB size=224x224 a...
4,VqX3u7446rqhpEjjCXX97w,food,<PIL.Image.Image image mode=RGB size=224x224 a...


In [29]:
np.array(df2_train.iloc[1, 2].getdata())

array([[243, 236, 228],
       [243, 236, 228],
       [243, 236, 228],
       ...,
       [ 98, 135,  47],
       [107, 142,  56],
       [105, 142,  54]])

# X train / test , y train / test

In [ ]:
series_X_train = df2_train.image.map(lambda img: np.array(img.getdata()).reshape(img_size, img_size, 3))
X_train = np.stack(series_X_train.to_numpy(copy=True))
X_train.shape